In [ ]:
# Importing necessary packages:
import re
import os
from glob import glob

import math
import numpy as np
import datetime
import pandas as pd
import datetime as dt

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rcParams, cycler
import matplotlib.lines as lines
from collections import OrderedDict
from matplotlib.legend import Legend
import matplotlib.patches as mpatches
import matplotlib.lines as mlines



import seaborn as sns

import scipy.integrate as integrate
import scipy.special as special

from sympy import elliptic_pi

from PipeLine import *

In [ ]:
bug_files = glob("../sumrule_data/N*-analyze_bug/N*.csv")
geometry = 'cylinder'
df_name = 'all_in_one-ens_avg'
ext=['-properties-ens_avg.csv']
ens_evg_properties_files = PipeLine.file_reader(bug_files,extensions=ext)
ens_avg_properties_df = PipeLine.df_of_properties(ens_evg_properties_files, df_name, geometry, to_file=False,index_col=0)
ens_avg_properties_df = ens_avg_properties_df.round(4)
ens_avg_properties_df['fsd_normalized'] = 0.0
ens_avg_properties_df['gyr_normalized'] = 0.0
ens_avg_properties_df['rflory_normalized'] = 0.0
ens_avg_properties_df['vfrc_c_normalized'] = (ens_avg_properties_df['dmon'] * ens_avg_properties_df['vfrc_crowd']) / ens_avg_properties_df['dcrowd'] 
unique_simulations = [list(input_set) for input_set in list(ens_avg_properties_df.groupby(['nmon','dcyl','dcrowd']).size().index)]
for nmon, dcyl, dcrowd in unique_simulations:
    condition = (ens_avg_properties_df['nmon'] == nmon) & (ens_avg_properties_df['dcyl'] == dcyl) & (ens_avg_properties_df['dcrowd'] == dcrowd) & (ens_avg_properties_df['vfrc_crowd'] == 0)
    fsd = ens_avg_properties_df[condition]['fsd'].values[0]
    gyr = ens_avg_properties_df[condition]['gyr'].values[0]
    rflory = ens_avg_properties_df[condition]['rflory'].values[0]
    cond_normal = (ens_avg_properties_df['nmon'] == nmon) & (ens_avg_properties_df['dcyl'] == dcyl) & (ens_avg_properties_df['dcrowd'] == dcrowd)
    ens_avg_properties_df.loc[cond_normal,'fsd_normalized'] = ens_avg_properties_df.loc[cond_normal,'fsd'] / fsd
    ens_avg_properties_df.loc[cond_normal,'gyr_normalized'] = ens_avg_properties_df.loc[cond_normal,'gyr'] / gyr
    ens_avg_properties_df.loc[cond_normal,'rflory_normalized'] = ens_avg_properties_df.loc[cond_normal,'rflory'] / rflory

ens_avg_properties_df['vfrc_crowd'] = ens_avg_properties_df['vfrc_crowd'].round(decimals=3)
ens_avg_properties_df['vfrc_crowd_eff'] = (np.pi * ens_avg_properties_df['dcrowd'] ** 3 / 6) * ens_avg_properties_df['ncrowd'] / ((np.pi / 4 * (ens_avg_properties_df['dcyl']-ens_avg_properties_df['dcrowd']) ** 2) * ens_avg_properties_df['lcyl'])
ens_avg_properties_df['vfrc_c_eff_normalized'] = (ens_avg_properties_df['dmon'] * ens_avg_properties_df['vfrc_crowd_eff']) / ens_avg_properties_df['dcrowd'] 
output = 'properties-ens_avg-all_in_one.csv'
ens_avg_properties_df.to_csv(output,index=False)

In [ ]:
#vfrc_crowds = list(ens_avg_properties_df.vfrc_crowd.sort_values().drop_duplicates())
#vfrc_colors = sns.color_palette("coolwarm", len(vfrc_crowds))
dcrowds = list(ens_avg_properties_df.dcrowd.sort_values().drop_duplicates())
print(dcrowds)
dcyls = list(ens_avg_properties_df.dcyl.sort_values().drop_duplicates())
print(dcyls)
nmons = list(ens_avg_properties_df.nmon.sort_values().drop_duplicates())
vfrc_crowds_selected = [0.1,0.2,0.3,0.4]

line_styles = ['--',':','-.',(0, (3, 1, 1, 1, 1, 1))]
ls_cycle = cycler('ls', line_styles) # equal to len(dcrowds)

line_widths = np.arange(1,3,1)
lw_cycle = cycler('lw', line_widths) # equal to len(nmons

marker_names = [r'$\frac{\phi_c(r)}{a_c}$',r'$\frac{\phi_m(r)}{a}$',r'$\sum\frac{\phi_i(r)}{a_i}$']
marker_styles = 'os^'
marker_cycle = cycler('marker',marker_styles)

vfrc_colors_selected = sns.color_palette("Set2", len(vfrc_crowds_selected))#,as_cmap=True)
#vfrc_colors_selected = sns.color_palette("hls", len(vfrc_crowds_selected))#,as_cmap=True)
vfrc_colors_selected

In [ ]:
xcol="vfrc_crowd"
df=ens_avg_properties_df
fontsize=20
colors = sns.color_palette(n_colors=4, palette="Set2", desat=1)
leg_labels = [r'$\frac{D}{a}$ (Line color)', '15.0', '20.0', '25.0','30.0', r'$N$ (Line size)', 1000, 2000, r'$\frac{a_c}{a}$ (Marker type)', '1.0', '2.0', '4.0']
PipeLine.chainsize_plot(ens_avg_properties_df, "vfrc_crowd_eff", leg_labels, colors, fontsize=20)
PipeLine.chainsize_plot(ens_avg_properties_df, "vfrc_crowd", leg_labels, colors, fontsize=20)
PipeLine.chainsize_plot(ens_avg_properties_df, "vfrc_c_eff_normalized", leg_labels, colors, fontsize=20)
PipeLine.chainsize_plot(ens_avg_properties_df, "vfrc_c_normalized", leg_labels, colors, fontsize=20)

In [ ]:
def organize_hists_ens_evg(pair_csvs,properties,xname,yname,xnrom,ynorm,filename=None):
    """
    This funcions receives a list of paths to histogram data (each contains two columns; one bins, other counts) in different directions and return a dict of paris of filenames and Pandas dataframes as pairs of key and values.
    """
    group_dfs = []
    properties[['groupname','garbage']] = properties.filename.str.split(pat='-',expand=True)
    for crd_csv, mon_csv in pair_csvs:
        crd_df = pd.read_csv(crd_csv,index_col=0)
        mon_df = pd.read_csv(mon_csv,index_col=0)
        group_name = list(crd_df.columns)[0].split('-')[0]
        
        cell_attrs = PipeLine.cellAttributes(group_name,warning=False)
        x_normalizer = {'dcyl':cell_attrs.dcyl, 'lcyl':cell_attrs.lcyl, 'degree': 180}
        y_normalizer = {'hist':1, 'rho':{'crd':1,'mon': 1}, 'phi': {'crd':(2 *  cell_attrs.ncrowd * cell_attrs.dcrowd ** 3) / (3 * cell_attrs.dcyl ** 2 * cell_attrs.lcyl),'mon': (2 *  cell_attrs.nmon * cell_attrs.dmon ** 3) / (3 * cell_attrs.dcyl ** 2 * cell_attrs.lcyl)}}
    
        crd_df = pd.read_csv(crd_csv,names=[xname,yname+'_c'],skiprows=1)
        mon_df = pd.read_csv(mon_csv,names=[yname+'_m'],skiprows=1,usecols=[1])
        
        crd_df[xname+'_norm'] = 2 * crd_df[xname] / x_normalizer[xnorm]
        crd_df[yname+'_norm'+'_c'] = crd_df[yname+'_c'] / y_normalizer[ynorm]['crd']
        crd_df[yname+'_norm'+'_c'] = crd_df[yname+'_norm'+'_c'] / crd_df[yname+'_norm'+'_c'].sum()
        

        mon_df[yname+'_norm'+'_m'] = mon_df[yname+'_m'] / y_normalizer[ynorm]['mon']
        mon_df[yname+'_norm'+'_m'] = mon_df[yname+'_norm'+'_m'] / mon_df[yname+'_norm'+'_m'].sum()
        
        # other properties:
        
        merged_df = pd.concat([crd_df,mon_df],axis=1)
        merged_df[yname+'_sum'] = (merged_df[yname+'_c'] + merged_df[yname+'_m']) / (y_normalizer[ynorm]['crd'] + y_normalizer[ynorm]['mon'])
        merged_df[yname+'_norm_sum'] = merged_df[yname+'_sum'] / merged_df[yname+'_sum'].sum()
        for col in properties.columns:
            cond = properties['groupname'] == group_name
            merged_df[col] = properties[cond][col].values[0]
        merged_df.drop(['filename','garbage'],axis=1,inplace=True)
        group_dfs.append(merged_df)
    
    df_of_hists = pd.concat(group_dfs)
    df_of_hists.reset_index(inplace=True,drop=True)
    if filename != None:
        outname = filename+'.csv'
        df_of_hists.to_csv(outname)
        
    return df_of_hists

In [ ]:
all_files = glob("../sumrule_data/*-analyze_all/N*.csv")
xname = 'r'
yname = 'phi_r'
xnorm = 'dcyl'
ynorm = 'phi'
ext = ['-rPhisCrd-ens_avg.csv','-rPhisMon-ens_avg.csv']
rPhis_ens_avg_csvs = PipeLine.file_reader(all_files,extensions=ext)
rPhis_ens_avg_df = organize_hists_ens_evg(rPhis_ens_avg_csvs,ens_avg_properties_df,xname,yname,xnorm,ynorm,filename='rPhis-ens_avg-all_in_one')

In [ ]:
rPhis_ens_avg_df.info()

In [ ]:
rPhis_df_sel_phis = rPhis_ens_avg_df[rPhis_ens_avg_df['vfrc_crowd'].isin([0.1,0.2,0.3,0.4])]

In [ ]:
xname='r_norm_c'
yname='phi_r'
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,12))
rcParams['axes.prop_cycle'] = cycler(color=vfrc_colors_selected) 
axes = axes.flatten()
fontsize=22
for nmon, lw in zip(nmons,lw_cycle):
    for dcyl, ls in zip(dcyls,ls_cycle):
        for idx, (dcrowd, axis) in enumerate(zip(dcrowds,axes)):
            axis.set_title(r'${a_c}/{a}=$'+'${}$'.format(dcrowd),fontsize=fontsize)
            axis.grid(True,ls=':',lw=1)
            axis.tick_params(axis ='both',direction='in')
            axis.set_ylabel(r'$\phi_m(r)$',fontsize=fontsize-3)
            axis.set_xlabel(r'${2r}/{D}$',fontsize=fontsize-3)
            PipeLine.xticks(axis,(0.0,1.0,0.2,0.02),code=True,fontsize=fontsize-6)
            lines = []
            for keyname, rPhi_ens_avg in rPhis_ens_avg_dfs.items():
                cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
                vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
                if cell_attrs.nmon == nmon: linew = lw
                if (cell_attrs.dcyl == dcyl) and (cell_attrs.dcrowd == dcrowd) and (vfrc_crowd_dummy in vfrc_crowds_selected):
                    line = axis.plot(xname,yname+'_c', data=rPhi_ens_avg, marker='o', fillstyle='none', markevery=10, markersize=7, alpha=0.8, **ls, **linew)
                    line = axis.plot(xname,yname+'_m', data=rPhi_ens_avg, marker='s', fillstyle='none', markevery=10, markersize=7, alpha=0.8, **ls, **linew)
                    line = axis.plot(xname,yname+'_sum', data=rPhi_ens_avg, marker='^', fillstyle='none', markevery=10, markersize=7, alpha=0.8, **ls, **linew)
                    lines += line

#PipeLine.yticks(axes[0],(0.0,0.030,0.005,0.0005),code=True,fontsize=fontsize-6,decimals=4)
#PipeLine.yticks(axes[1],(0.0,0.030,0.006,0.0006),code=True,fontsize=fontsize-6,decimals=4)
#PipeLine.yticks(axes[2],(0.0,0.015,0.003,0.0003),code=True,fontsize=fontsize-6,decimals=4)
#PipeLine.yticks(axes[3],(0.0,0.015,0.003,0.0003),code=True,fontsize=fontsize-6,decimals=4)

#axes[0].set_ylabel(r'$\phi_c(r)$',fontsize=fontsize)
#axes[2].set_ylabel(r'$\phi_c(r)$',fontsize=fontsize)
#axis.set_xlabel(r'$\frac{2r}{D}$',fontsize=fontsize)
color_patches = PipeLine.color_patcher(vfrc_colors_selected)
color_legend = Legend(axes[0],handles=color_patches,labels= vfrc_crowds_selected,title=r'$\phi_c$ (color)',ncol=1,
                      title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False,loc='upper right')#,bbox_to_anchor=(1.02,-0.02))
axes[0].add_artist(color_legend)

ls_handles = PipeLine.ls_handler(dcyls,line_styles)
axes[1].legend(handles=ls_handles, title=r'${D}/{a}$ (line style)', ncol=1, title_fontsize=fontsize-4,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

lw_handles = PipeLine.lw_handler(nmons,line_widths)
axes[2].legend(handles=lw_handles, title=r'$N$ (line width)', ncol=1, title_fontsize=fontsize-4,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

marker_handles = PipeLine.marker_handler(marker_names,marker_styles,fillstyle='none', markersize=10)
axes[3].legend(handles=marker_handles, title=r'Local volume fraction (marker style)', ncol=3, title_fontsize=fontsize-4,
               fontsize=fontsize-4, framealpha=None, frameon=False, loc='upper right')



fig.tight_layout()
plt.tight_layout()
today = dt.date.today().strftime('%Y%m%d')
fname = "phic_local-grid_dcrowd-"+yname+'-'+today+".pdf"
plt.savefig(fname,dpi=200)

In [ ]:
xname='r_norm_c'
yname='phi_r_norm'
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,12))
rcParams['axes.prop_cycle'] = cycler(color=vfrc_colors_selected) 
axes = axes.flatten()
fontsize=22
for nmon, lw in zip(nmons,lw_cycle):
    for dcrowd, ls in zip(dcrowds,ls_cycle):
        for idx, (dcyl, axis) in enumerate(zip(dcyls,axes)):
            axis.set_title(r'${D}/{a}=$'+'${}$'.format(dcyl),fontsize=fontsize)
            axis.grid(True,ls=':',lw=1)
            axis.tick_params(axis ='both',direction='in')
            #if idx > 1:
             #   PipeLine.yticks(axis,(0.0,0.015,0.003,0.0006),code=True,fontsize=fontsize-6)
            #elif idx == 1: 
             #   PipeLine.yticks(axis,(0.0,0.02,0.004,0.0008),code=True,fontsize=fontsize-6)
            #else:
             #   PipeLine.yticks(axis,(0.0,0.03,0.006,0.0012),code=True,fontsize=fontsize-6)

            axis.set_ylabel(r'$\phi_m(r)$',fontsize=fontsize-3)
            axis.set_xlabel(r'${2r}/{D}$',fontsize=fontsize-3)
            PipeLine.xticks(axis,(0.0,1.0,0.2,0.02),code=True,fontsize=fontsize-6)
            lines = []
            for keyname, rPhi_ens_avg in rPhis_ens_avg_dfs.items():
                cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
                vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
                if cell_attrs.nmon == nmon: linew = lw
                if (cell_attrs.dcyl == dcyl) and (cell_attrs.dcrowd == dcrowd) and (vfrc_crowd_dummy in vfrc_crowds_selected):
                    line = axis.plot(xname,yname+'_c', data=rPhi_ens_avg, marker='o', fillstyle='none', markevery=10, markersize=5, alpha=0.8, **ls, **linew)
                    line = axis.plot(xname,yname+'_m', data=rPhi_ens_avg, marker='s', fillstyle='none', markevery=10, markersize=5, alpha=0.8, **ls, **linew)
                    line = axis.plot(xname,yname+'_sum', data=rPhi_ens_avg, marker='^', fillstyle='none', markevery=10, markersize=5, alpha=0.8, **ls, **linew)
                    lines += line
#axes[0].set_ylabel(r'$\phi_c(r)$',fontsize=fontsize)
#axes[2].set_ylabel(r'$\phi_c(r)$',fontsize=fontsize)
#axis.set_xlabel(r'$\frac{2r}{D}$',fontsize=fontsize)
color_patches = PipeLine.color_patcher(vfrc_colors_selected)
color_legend = Legend(axes[0],handles=color_patches,labels= vfrc_crowds_selected,title=r'$\phi_c$ (color)',ncol=1,
                      title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False,loc='upper right')#,bbox_to_anchor=(1.02,-0.02))
axes[0].add_artist(color_legend)

ls_handles = PipeLine.ls_handler(dcrowds,line_styles)
axes[1].legend(handles=ls_handles, title=r'${D}/{a}$ (line style)', ncol=1, title_fontsize=fontsize-4,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

lw_handles = PipeLine.lw_handler(nmons,line_widths)
axes[2].legend(handles=lw_handles, title=r'$N$ (line width)', ncol=1, title_fontsize=fontsize-4,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

marker_handles = PipeLine.marker_handler(marker_names,marker_styles,fillstyle='none', markersize=10)
axes[3].legend(handles=marker_handles, title=r'Local volume fraction (marker style)', ncol=3, title_fontsize=fontsize-4,
               fontsize=fontsize-4, framealpha=None, frameon=False, loc='upper right')

fig.tight_layout()
plt.tight_layout()
today = dt.date.today().strftime('%Y%m%d')
fname = "phic_local-grid_dcyl-"+yname+'-'+today+".pdf"
plt.savefig(fname,dpi=200)

## The sum rule in the radial direction:

### 1. $a_c$ as subplots

In [ ]:
xname='r_norm_c'
yname='phi_r_norm_sum'
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,12),sharey='row')
rcParams['axes.prop_cycle'] = cycler(color=vfrc_colors_selected) 
axes = axes.flatten()
fontsize=22
for nmon, lw in zip(nmons,lw_cycle):
    for dcyl, ls in zip(dcyls,ls_cycle):
        for idx, (dcrowd, axis) in enumerate(zip(dcrowds,axes)):
            axis.set_title(r'${a_c}/{a}=$'+'${}$'.format(dcrowd),fontsize=fontsize)
            axis.grid(True,ls=':',lw=1)
            axis.tick_params(axis ='both',direction='in')
            #axis.set_ylabel(r'$\phi_m(r)$',fontsize=fontsize)
            if idx % 2 == 0: axis.set_ylabel(r'$\sum{\phi_i(r)}/{a_i}$',fontsize=fontsize-3)
            axis.set_xlabel(r'${2r}/{D}$',fontsize=fontsize-3)
            PipeLine.xticks(axis,(0.0,1.0,0.2,0.02),code=True,fontsize=fontsize-6)
            
            for keyname, rPhi_ens_avg in rPhis_ens_avg_dfs.items():
                direction = rPhi_ens_avg[xname]
                cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
                vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
                if cell_attrs.nmon == nmon: linew = lw
                if (cell_attrs.dcyl == dcyl) and (cell_attrs.dcrowd == dcrowd) and (vfrc_crowd_dummy in vfrc_crowds_selected):
                    line = axis.plot(xname,yname, data=rPhi_ens_avg, fillstyle='none', markevery=10, markersize=5, alpha=0.9, **ls, **linew)
                    lines += line
#PipeLine.yticks(axes[0],(0.0,0.02,0.004,0.0004),code=True,fontsize=fontsize-6,decimals=4)
#PipeLine.yticks(axes[1],(0.0,0.02,0.004,0.0004),code=True,fontsize=fontsize-6,decimals=4)
#PipeLine.yticks(axes[2],(0.0,0.012,0.003,0.0003),code=True,fontsize=fontsize-6,decimals=4)
#PipeLine.yticks(axes[3],(0.0,0.012,0.003,0.0003),code=True,fontsize=fontsize-6,decimals=4)

#axes[0].set_ylabel(r'$\phi_c(r)$',fontsize=fontsize)
#axes[2].set_ylabel(r'$\phi_c(r)$',fontsize=fontsize)
#axis.set_xlabel(r'$\frac{2r}{D}$',fontsize=fontsize)
color_patches = PipeLine.color_patcher(vfrc_colors_selected)
color_legend = Legend(axes[0],handles=color_patches,labels= vfrc_crowds_selected,title=r'$\phi_c$ (color)',ncol=1,
                      title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False,loc='upper right')#,bbox_to_anchor=(1.02,-0.02))
axes[0].add_artist(color_legend)

ls_handles = PipeLine.ls_handler(dcyls,line_styles)
axes[1].legend(handles=ls_handles, title=r'${D}/{a}$ (line style)', ncol=1, title_fontsize=fontsize-4,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

lw_handles = PipeLine.lw_handler(nmons,line_widths)
axes[2].legend(handles=lw_handles, title=r'$N$ (line width)', ncol=1, title_fontsize=fontsize-4,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

fig.tight_layout()
plt.tight_layout()
today = dt.date.today().strftime('%Y%m%d')
fname = "phic_local-grid_dcrowd-only-"+yname+'-'+today+".pdf"
plt.savefig(fname,dpi=200)

### 2. $D$ as sub-plots

In [ ]:
xname='r_norm_c'
yname='phi_r_norm_sum'
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,12),sharey='row')
rcParams['axes.prop_cycle'] = cycler(color=vfrc_colors_selected) 
axes = axes.flatten()
fontsize=22
for nmon, lw in zip(nmons,lw_cycle):
    for dcrowd, ls in zip(dcrowds,ls_cycle):
        for idx, (dcyl, axis) in enumerate(zip(dcyls,axes)):
            axis.set_title(r'${D}/{a}=$'+'${}$'.format(dcyl),fontsize=fontsize)
            axis.grid(True,ls=':',lw=1)
            axis.tick_params(axis ='both',direction='in')
            if idx > 1:
                PipeLine.yticks(axis,(0.0,0.012,0.003,0.0003),code=True,fontsize=fontsize-6)
            else:
                PipeLine.yticks(axis,(0.0,0.020,0.004,0.0004),code=True,fontsize=fontsize-6)

            if idx % 2 == 0: axis.set_ylabel(r'$\sum{\phi_i(r)}/{a_i}$',fontsize=fontsize-3)
            axis.set_xlabel(r'${2r}/{D}$',fontsize=fontsize-3)
            PipeLine.xticks(axis,(0.0,1.0,0.2,0.02),code=True,fontsize=fontsize-6)
            lines = []
            for keyname, rPhi_ens_avg in rPhis_ens_avg_dfs.items():
                cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
                vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
                if cell_attrs.nmon == nmon: linew = lw
                if (cell_attrs.dcyl == dcyl) and (cell_attrs.dcrowd == dcrowd) and (vfrc_crowd_dummy in vfrc_crowds_selected):
                    line = axis.plot(xname,yname, data=rPhi_ens_avg, fillstyle='none', markevery=10, markersize=5, alpha=0.8, **ls, **linew)
                    lines += line
#axes[0].set_ylabel(r'$\phi_c(r)$',fontsize=fontsize)
#axes[2].set_ylabel(r'$\phi_c(r)$',fontsize=fontsize)
#axis.set_xlabel(r'$\frac{2r}{D}$',fontsize=fontsize)
color_patches = PipeLine.color_patcher(vfrc_colors_selected)
color_legend = Legend(axes[0],handles=color_patches,labels= vfrc_crowds_selected,title=r'$\phi_c$ (color)',ncol=1,
                      title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False,loc='upper right')#,bbox_to_anchor=(1.02,-0.02))
axes[0].add_artist(color_legend)
ls_handles = PipeLine.ls_handler(dcrowds,line_styles)
axes[1].legend(handles=ls_handles, title=r'${a_c}/{a}$ (line style)', ncol=1, title_fontsize=fontsize-4,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

lw_handles = PipeLine.lw_handler(nmons,line_widths)
axes[2].legend(handles=lw_handles, title=r'$N$ (line width)', ncol=1, title_fontsize=fontsize-4,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

fig.tight_layout()
plt.tight_layout()
today = dt.date.today().strftime('%Y%m%d')
fname = "phic_local-grid_dcyl-only-"+yname+'-'+today+".pdf"
plt.savefig(fname,dpi=200)

### 3. $\phi_c$ as sub-plots

In [ ]:
#vfrc_crowds = list(ens_avg_properties_df.vfrc_crowd.sort_values().drop_duplicates())
#vfrc_colors = sns.color_palette("coolwarm", len(vfrc_crowds))
dcrowds = list(ens_avg_properties_df.dcrowd.sort_values().drop_duplicates())
print(dcrowds)
dcyls = list(ens_avg_properties_df.dcyl.sort_values().drop_duplicates())
print(dcyls)
nmons = list(ens_avg_properties_df.nmon.sort_values().drop_duplicates())
vfrc_crowds_selected = [0.1,0.2,0.3,0.4]

line_styles = ['--',':','-.',(0, (3, 1, 1, 1, 1, 1))]
ls_cycle = cycler('ls', line_styles) # equal to len(dcrowds)

line_widths = np.arange(1,3,1)
lw_cycle = cycler('lw', line_widths) # equal to len(nmons

marker_names = [r'$\frac{\phi_c(r)}{a_c}$',r'$\frac{\phi_m(r)}{a}$',r'$\sum\frac{\phi_i(r)}{a_i}$']
marker_styles = 'os^'
marker_cycle = cycler('marker',marker_styles)

dcrowd_colors_selected = sns.color_palette("Set2", len(dcrowds))#,as_cmap=True)
#vfrc_colors_selected = sns.color_palette("hls", len(vfrc_crowds_selected))#,as_cmap=True)
vfrc_colors_selected

In [ ]:
xname='r_norm_c'
yname='phi_r_norm_sum'
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,12),sharey='row')
rcParams['axes.prop_cycle'] = cycler(color=dcrowd_colors_selected) 
axes = axes.flatten()
fontsize=22
for nmon, lw in zip(nmons,lw_cycle):
    for dcyl, ls in zip(dcyls,ls_cycle):
        for idx, (vfrc_crowd, axis) in enumerate(zip(vfrc_crowds_selected,axes)):
            axis.set_title(r'$\phi_c=$'+'${}$'.format(vfrc_crowd),fontsize=fontsize)
            axis.grid(True,ls=':',lw=1)
            axis.tick_params(axis ='both',direction='in')
            #axis.set_ylabel(r'$\phi_m(r)$',fontsize=fontsize)
            if idx % 2 == 0: axis.set_ylabel(r'$\sum{\phi_i(r)}/{a_i}$',fontsize=fontsize-3)
            axis.set_xlabel(r'${2r}/{D}$',fontsize=fontsize-3)
            PipeLine.xticks(axis,(0.0,1.0,0.2,0.02),code=True,fontsize=fontsize-6)
            
            for keyname, rPhi_ens_avg in rPhis_ens_avg_dfs.items():
                direction = rPhi_ens_avg[xname]
                cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
                vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
                if cell_attrs.nmon == nmon: linew = lw
                if (cell_attrs.dcyl == dcyl) and (vfrc_crowd_dummy == vfrc_crowd) and (cell_attrs.dcrowd in dcrowds):
                    line = axis.plot(xname,yname, data=rPhi_ens_avg, fillstyle='none', markevery=10, markersize=5, alpha=0.9, **ls, **linew)
                    lines += line
#PipeLine.yticks(axes[0],(0.0,0.02,0.004,0.0004),code=True,fontsize=fontsize-6,decimals=4)
#PipeLine.yticks(axes[1],(0.0,0.02,0.004,0.0004),code=True,fontsize=fontsize-6,decimals=4)
#PipeLine.yticks(axes[2],(0.0,0.012,0.003,0.0003),code=True,fontsize=fontsize-6,decimals=4)
#PipeLine.yticks(axes[3],(0.0,0.012,0.003,0.0003),code=True,fontsize=fontsize-6,decimals=4)

#axes[0].set_ylabel(r'$\phi_c(r)$',fontsize=fontsize)
#axes[2].set_ylabel(r'$\phi_c(r)$',fontsize=fontsize)
#axis.set_xlabel(r'$\frac{2r}{D}$',fontsize=fontsize)
color_patches = PipeLine.color_patcher(dcrowd_colors_selected)
color_legend = Legend(axes[0],handles=color_patches,labels= dcrowds,title=r'$a_c$ (color)',ncol=1,
                      title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False,loc='upper right')#,bbox_to_anchor=(1.02,-0.02))
axes[0].add_artist(color_legend)

ls_handles = PipeLine.ls_handler(dcyls,line_styles)
axes[1].legend(handles=ls_handles, title=r'${D}/{a}$ (line style)', ncol=1, title_fontsize=fontsize-4,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

lw_handles = PipeLine.lw_handler(nmons,line_widths)
axes[2].legend(handles=lw_handles, title=r'$N$ (line width)', ncol=1, title_fontsize=fontsize-4,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

fig.tight_layout()
plt.tight_layout()
today = dt.date.today().strftime('%Y%m%d')
fname = "phic_local-grid_vftc_c-only-"+yname+'-'+today+".pdf"
plt.savefig(fname,dpi=200)

In [ ]:
def sumrule_line(vfrc_cs,dcrowds,direction):
    sumrule = [vfrc_c*dcrowd for vfrc_c, dcrowd in zip([vfrc_cs],[dcrowds])]
    sumrule = np.ones((len(direction),2)) * sumrule
    sumrule[:,0] = direction
    return sumrule
jafar = sumrule_line(0.3,2,rPhi_ens_avg['r_norm_c'].to_numpy())
fig, axis = plt.subplots(nrows=1,ncols=1,figsize=(16,12),sharey='row')
axis.plot(jafar[:,0],jafar[:,1])

In [ ]:
xname='r_norm_c'
yname='phi_r_norm_sum'
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,12),sharey='row')
rcParams['axes.prop_cycle'] = cycler(color=vfrc_colors_selected) 
axes = axes.flatten()
fontsize=22
for nmon, lw in zip(nmons,lw_cycle):
    for dcrowd, ls in zip(dcrowds,ls_cycle):
        for idx, (dcyl, axis) in enumerate(zip(dcyls,axes)):
            axis.set_title(r'${D}/{a}=$'+'${}$'.format(dcyl),fontsize=fontsize)
            axis.grid(True,ls=':',lw=1)
            axis.tick_params(axis ='both',direction='in')
            if idx > 1:
                PipeLine.yticks(axis,(0.0,0.012,0.003,0.0003),code=True,fontsize=fontsize-6)
            else:
                PipeLine.yticks(axis,(0.0,0.020,0.004,0.0004),code=True,fontsize=fontsize-6)

            if idx % 2 == 0: axis.set_ylabel(r'$\sum{\phi_i(r)}/{a_i}$',fontsize=fontsize-3)
            axis.set_xlabel(r'${2r}/{D}$',fontsize=fontsize-3)
            PipeLine.xticks(axis,(0.0,1.0,0.2,0.02),code=True,fontsize=fontsize-6)
            lines = []
            for keyname, rPhi_ens_avg in rPhis_ens_avg_dfs.items():
                cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
                vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
                if cell_attrs.nmon == nmon: linew = lw
                if (cell_attrs.dcyl == dcyl) and (cell_attrs.dcrowd == dcrowd) and (vfrc_crowd_dummy in vfrc_crowds_selected):
                    line = axis.plot(xname,yname, data=rPhi_ens_avg, fillstyle='none', markevery=10, markersize=5, alpha=0.8, **ls, **linew)
                    lines += line
#axes[0].set_ylabel(r'$\phi_c(r)$',fontsize=fontsize)
#axes[2].set_ylabel(r'$\phi_c(r)$',fontsize=fontsize)
#axis.set_xlabel(r'$\frac{2r}{D}$',fontsize=fontsize)
color_patches = PipeLine.color_patcher(vfrc_colors_selected)
color_legend = Legend(axes[0],handles=color_patches,labels= vfrc_crowds_selected,title=r'$\phi_c$ (color)',ncol=1,
                      title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False,loc='upper right')#,bbox_to_anchor=(1.02,-0.02))
axes[0].add_artist(color_legend)
ls_handles = PipeLine.ls_handler(dcrowds,line_styles)
axes[1].legend(handles=ls_handles, title=r'${a_c}/{a}$ (line style)', ncol=1, title_fontsize=fontsize-4,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

lw_handles = PipeLine.lw_handler(nmons,line_widths)
axes[2].legend(handles=lw_handles, title=r'$N$ (line width)', ncol=1, title_fontsize=fontsize-4,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

fig.tight_layout()
plt.tight_layout()
today = dt.date.today().strftime('%Y%m%d')
fname = "phic_local-grid_dcyl-only-"+yname+'-'+today+".pdf"
plt.savefig(fname,dpi=200)

In [ ]:
all_files = glob("../sumrule_data/*ac2.0-analyze_all/N*.csv") + glob("../sumrule_data/*ac4.0-analyze_all/N*.csv")
xname = 'r'
yname = 'phi_r'
xnorm = 'dcyl'
ynorm = 'phi'
ext=['-rPhisCrd-ens_avg.csv']
rPhisCrd_ens_avg_csvs = PipeLine.file_reader(all_files,extensions=ext)
ext=['-rPhisMon-ens_avg.csv']
rPhisMon_ens_avg_csvs = PipeLine.file_reader(all_files,extensions=ext)
rPhis_ens_avg_dfs = organize_hists_ens_evg(rPhisCrd_ens_avg_csvs,rPhisMon_ens_avg_csvs,xname,yname,xnorm,ynorm)

dcyls = [25.0,20.0,15.0,30.0]
dcrowds = [2.0,2.0,2.0,4.0]

In [ ]:
xname='r_norm_c'
yname='phi_r_norm'
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,12))
rcParams['axes.prop_cycle'] = cycler(color=vfrc_colors_selected) 
axes = axes.flatten()
fontsize=22

for idx, (dcrowd, dcyl, axis) in enumerate(zip(dcrowds,dcyls,axes)):
    axis.set_title(r'${D}/{a}=$'+'${}$'.format(dcyl)+'$,{a_c}/{a}=$'+'${}$'.format(dcrowd)+r': $\kappa=$'+'${:.2f}$'.format(dcrowd/(dcyl-dcrowd)),fontsize=fontsize)
    axis2 = axis.twinx()
    axis2.set_ylabel(r'$\sum{\phi_i(r)}/{a_i}$',fontsize=fontsize-3)
    axis.grid(True,ls=':',lw=1)
    axis.tick_params(axis ='both',direction='in')

    if idx == 0:
        PipeLine.yticks(axis,(0.0,0.015,0.003,0.0006),code=True,fontsize=fontsize-6)
        PipeLine.yticks(axis2,(0.0,0.015,0.003,0.0006),code=True,fontsize=fontsize-6)
    elif idx == 1: 
        PipeLine.yticks(axis,(0.0,0.02,0.004,0.0008),code=True,fontsize=fontsize-6)
        PipeLine.yticks(axis2,(0.0,0.02,0.004,0.0008),code=True,fontsize=fontsize-6)
    elif idx == 2: 
        PipeLine.yticks(axis,(0.0,0.03,0.005,0.001),code=True,fontsize=fontsize-6)
        PipeLine.yticks(axis2,(0.0,0.03,0.005,0.001),code=True,fontsize=fontsize-6)  
    else:
        PipeLine.yticks(axis,(0.0,0.015,0.003,0.0006),code=True,fontsize=fontsize-6)
        PipeLine.yticks(axis2,(0.0,0.015,0.003,0.0006),code=True,fontsize=fontsize-6) 
    axis.set_ylabel(r'$\phi_i(r)$',fontsize=fontsize-3)
    axis.set_xlabel(r'${2r}/{D}$',fontsize=fontsize-3)
    PipeLine.xticks(axis,(0.0,1.0,0.2,0.02),code=True,fontsize=fontsize-6)


    lines = []
    for keyname, rPhi_ens_avg in rPhis_ens_avg_dfs.items():
        cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
        vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
        if (cell_attrs.dcyl == dcyl) and (cell_attrs.dcrowd == dcrowd) and (vfrc_crowd_dummy in vfrc_crowds_selected):
            line = axis.plot(xname,yname+'_c', data=rPhi_ens_avg, marker='o', fillstyle='none', markevery=10, markersize=10, alpha=1,ls=':')
            line = axis.plot(xname,yname+'_m', data=rPhi_ens_avg, marker='s', fillstyle='none', markevery=10, markersize=10, alpha=1,ls='--')
            line = axis.plot(xname,yname+'_sum', data=rPhi_ens_avg, marker='^', fillstyle='none', markevery=10, markersize=10, alpha=1)
            lines += line

color_patches = PipeLine.color_patcher(vfrc_colors_selected)
color_legend = Legend(axes[0],handles=color_patches,labels= vfrc_crowds_selected,title=r'$\phi_c$ (color)',ncol=2,
                      title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False,loc='upper right')#,bbox_to_anchor=(1.02,-0.02))
axes[0].add_artist(color_legend)

marker_handles = PipeLine.marker_handler(marker_names,marker_styles,fillstyle='none',markersize=10)
axes[3].legend(handles=marker_handles, title=r'Local volume fraction (marker style)', ncol=3, title_fontsize=fontsize-6,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

fig.tight_layout()
plt.tight_layout()
today = dt.date.today().strftime('%Y%m%d')
fname = "phic_local-grid_dcyl-all-selected_crowd-"+yname+'-'+today+".pdf"
plt.savefig(fname,dpi=200)

In [ ]:
all_files = glob("../sumrule_data/*D25.0ac*-analyze_all/N*.csv") + glob("../sumrule_data/*D30.0ac6.0-analyze_all/N*.csv")
xname = 'r'
yname = 'phi_r'
xnorm = 'dcyl'
ynorm = 'phi'
ext=['-rPhisCrd-ens_avg.csv']
rPhisCrd_ens_avg_csvs = PipeLine.file_reader(all_files,extensions=ext)
ext=['-rPhisMon-ens_avg.csv']
rPhisMon_ens_avg_csvs = PipeLine.file_reader(all_files,extensions=ext)
rPhis_ens_avg_dfs = organize_hists_ens_evg(rPhisCrd_ens_avg_csvs,rPhisMon_ens_avg_csvs,xname,yname,xnorm,ynorm)

dcyls = [25.0,25.0,25.0,30.0]
dcrowds = [1.0,2.0,4.0,6.0]

In [ ]:
xname='r_norm_c'
yname='phi_r_norm'
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,12))
rcParams['axes.prop_cycle'] = cycler(color=vfrc_colors_selected) 
axes = axes.flatten()
fontsize=22


for idx, (dcrowd, dcyl, axis) in enumerate(zip(dcrowds,dcyls,axes)):
    axis.set_title(r'${D}/{a}=$'+'${}$'.format(dcyl)+'$,{a_c}/{a}=$'+'${}$'.format(dcrowd)+r': $\kappa=$'+'${:.2f}$'.format(dcrowd/(dcyl-dcrowd)),fontsize=fontsize)
    axis2 = axis.twinx()
    axis2.set_ylabel(r'$\sum{\phi_i(r)}/{a_i}$',fontsize=fontsize-3)
    axis.grid(True,ls=':',lw=1)
    axis.tick_params(axis ='both',direction='in')

    if idx == 0:
        PipeLine.yticks(axis,(0.0,0.015,0.003,0.0003),code=True,fontsize=fontsize-6)
        PipeLine.yticks(axis2,(0.0,0.015,0.003,0.0003),code=True,fontsize=fontsize-6)
    elif idx == 1: 
        PipeLine.yticks(axis,(0.0,0.015,0.003,0.0003),code=True,fontsize=fontsize-6)
        PipeLine.yticks(axis2,(0.0,0.015,0.003,0.0003),code=True,fontsize=fontsize-6)
    elif idx == 2: 
        PipeLine.yticks(axis,(0.0,0.015,0.003,0.0003),code=True,fontsize=fontsize-6)
        PipeLine.yticks(axis2,(0.0,0.015,0.003,0.0003),code=True,fontsize=fontsize-6)  
    else:
        PipeLine.yticks(axis,(0.0,0.015,0.003,0.0003),code=True,fontsize=fontsize-6)
        PipeLine.yticks(axis2,(0.0,0.015,0.003,0.0003),code=True,fontsize=fontsize-6) 
    axis.set_ylabel(r'$\phi_i(r)$',fontsize=fontsize-3)
    axis.set_xlabel(r'${2r}/{D}$',fontsize=fontsize-3)
    PipeLine.xticks(axis,(0.0,1.0,0.2,0.02),code=True,fontsize=fontsize-6)


    lines = []
    for keyname, rPhi_ens_avg in rPhis_ens_avg_dfs.items():
        cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
        vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
        if (cell_attrs.dcyl == dcyl) and (cell_attrs.dcrowd == dcrowd) and (vfrc_crowd_dummy in vfrc_crowds_selected):
            line = axis.plot(xname,yname+'_c', data=rPhi_ens_avg, marker='o', fillstyle='none', markevery=10, markersize=10, alpha=1,ls=':')
            line = axis.plot(xname,yname+'_m', data=rPhi_ens_avg, marker='s', fillstyle='none', markevery=10, markersize=10, alpha=1,ls='--')
            line = axis.plot(xname,yname+'_sum', data=rPhi_ens_avg, marker='^', fillstyle='none', markevery=10, markersize=10, alpha=1)
            lines += line

color_patches = PipeLine.color_patcher(vfrc_colors_selected)
color_legend = Legend(axes[0],handles=color_patches,labels= vfrc_crowds_selected,title=r'$\phi_c$ (color)',ncol=2,
                      title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False,loc='upper right')#,bbox_to_anchor=(1.02,-0.02))
axes[0].add_artist(color_legend)

marker_handles = PipeLine.marker_handler(marker_names,marker_styles,fillstyle='none',markersize=10)
axes[3].legend(handles=marker_handles, title=r'Local volume fraction (marker style)', ncol=3, title_fontsize=fontsize-6,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

fig.tight_layout()
plt.tight_layout()
today = dt.date.today().strftime('%Y%m%d')
fname = "phic_local-grid_dcrowd-all-selected_dcyl-"+yname+'-'+today+".pdf"
plt.savefig(fname,dpi=200)